# This notebook is for creating dimension table for sales table usin SQL  - as part of the pipline

In [1]:
df_sales =  spark.read.table("dbo.sales")
display(df_sales)

StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3e2edf66-6ce7-4428-9d05-8dcbe0498763)

creating temporary view

In [5]:
df_sales.createOrReplaceTempView("df_new")

StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 7, Finished, Available, Finished)

# Creating Date dimension table

In [15]:
%%sql
CREATE TABLE DimDate2 AS
SELECT
    ROW_NUMBER() OVER (ORDER BY OrderDate) AS DateId,  -- sequential ID
    CAST(OrderDate AS DATE) AS Date,
    YEAR(OrderDate) AS Year,
    MONTH(OrderDate) AS Month
FROM df_new

StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Adding Date ID

# Creating Customer Dimension Table

In [19]:
%%sql 
CREATE TABLE DimCustomer3 AS 
SELECT 
Row_Number() OVER (ORDER BY OrderDate) AS CustomerID, -- sequential customer ID
CustomerName, EmailAddress 
From df_new



StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 21, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

# Create sales Fact table 

In [20]:
%%sql 
CREATE TABLE Factsales2 AS 
SELECT UnitPrice, Quantity, TaxAmount, CustomerName, Orderdate
from df_new

StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

# In order to build the semantic mode we need to have the primary key of date and customer ID in the fact table as foreign key - so I will use join query to complete this task

In [26]:
%%sql
CREATE TABLE NEWfactsales AS 
Select s.UnitPrice, s.Quantity, s.TaxAmount, c.CustomerID, d.DateId
from Factsales2 as s 
Join DimCustomer3 as c
ON s.CustomerName = c.CustomerName
Join DimDate2 as d
on s.Orderdate = d.`Date`

StatementMeta(, 3b52c96e-d0f3-47ec-a655-2405cfee256d, 28, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>